In [ ]:
import numpy as np
import scipy.misc
from PIL import Image
import scipy.io
import os
import cv2

# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/rcf/
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe


In [ ]:
im = Image.open('input_image_root/filename.png)
in_ = np.array(im, dtype=np.float32)
in_ = in_[:,:,::-1]
in_ -= np.array((104.00698793,116.66876762,122.67891434))

# load net
net = caffe.Net('test.prototxt', 'rcf_pretrained_bsds.caffemodel', caffe.TEST)


sz1 = np.floor(in_.shape[0]/4)
sz2 = np.floor(in_.shape[1]/4)
E = np.zeros((in_.shape[0], in_.shape[1]))

C = 10
for wini in range(0,8):
    for winj in range(0,8):
        i = wini/2; j = winj/2;
        print winj
        if (i+1)*sz1 > in_.shape[0]:
            winsizeimax = in_.shape[0]
        else :
            winsizeimax = (i+1)*sz1
        if (j+1)*sz2 > in_.shape[1]:
            winsizejmax = in_.shape[1]
        else :
            winsizejmax = (j+1)*sz2

        wiL = int(i*sz1)
        wiU = int(winsizeimax)
        wjL = int(j*sz2)
        wjU = int(winsizejmax)
        wi5L = int(max(0,i*sz1-C))
        wi5U = int(min(winsizeimax+C, in_.shape[0]))
        wj5L = int(max(0,j*sz2-C))
        wj5U = int(min(winsizejmax+C, in_.shape[1]))

        im1_ = (in_[wi5L:wi5U, wj5L:wj5U,:])
        im_ = im1_.transpose((2, 0, 1))
        net.blobs['data'].reshape(1, *im_.shape)
        net.blobs['data'].data[...] = im_
        # run net and take argmax for prediction
        net.forward()
        fuse = net.blobs['sigmoid-fuse'].data[0][0, :, :]

        EiL = wiL - wi5L;
        EjL = wjL - wj5L;
        EiU = EiL + wiU - wiL
        EjU = EjL + wjU - wjL

        E[wiL:wiU, wjL:wjU] = E[wiL:wiU, wjL:wjU]  + fuse[EiL:EiU,EjL:EjU]

# cv2.imwrite('output_root/filename.png', E/4)

In [ ]:
# check the output
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(E/4, cmap='gray')